<a href="https://colab.research.google.com/github/angellmethod/DogFaceNet/blob/John/VGG16Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install deepface

     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 2.3 MB 8.9 MB/s 
     |████████████████████████████████| 462 kB 36.4 MB/s 


In [ ]:
% cd drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
from deepface import DeepFace

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [ ]:
#df = DeepFace.find(img_path = "w210-capstone/data/after_4/0/0.0.png", db_path = "w210-capstone/data/after_4/", enforce_detection=False)

In [ ]:
#df.sort_values(by=['VGG-Face_cosine'])

In [ ]:
#! wget https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5

In [ ]:
model = DeepFace.build_model('VGG-Face')

vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [01:43<00:00, 5.59MB/s]


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_input (Input  [(None, 224, 224, 3)]    0         
 Layer)                                                          
                                                                 
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                             

In [ ]:
import tensorflow as tf

In [ ]:
#model.layers[-2].get_config()

In [ ]:
model_reduced = tf.keras.models.Sequential()

for layer in model.layers[:-2]:
  model_reduced.add(layer)

In [ ]:
#import numpy as np

In [ ]:
#model_reduced.layers[1].get_weights()

In [ ]:
#model.layers[2].get_weights()

In [ ]:
model_reduced.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

In [ ]:
#model_reduced.layers[2].get_weights()

In [ ]:
model_reduced.add(tf.keras.layers.Convolution2D(1393, (1, 1)))
model_reduced.add(tf.keras.layers.Flatten())
model_reduced.add(tf.keras.layers.Activation('softmax'))

In [ ]:
model_reduced.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

In [ ]:
for layer in model_reduced.layers[:-3]:
  layer.trainable = False

In [ ]:
#for layer in model_reduced.layers[:]:
#  print(layer.trainable)

In [ ]:
# ! ls

In [ ]:
import pathlib
data_dir = 'data/dogfacenet/aligned/after_4_bis'
data_dir = pathlib.Path(data_dir)

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(224, 224))

Found 8363 files belonging to 1393 classes.
Using 6691 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(224, 224))

Found 8363 files belonging to 1393 classes.
Using 1672 files for validation.


In [ ]:
#Just copied from here
#https://stackoverflow.com/questions/61552475/properly-set-up-exponential-decay-of-learning-rate-in-tensorflow
#because keras docs did not explain anything about how to pick a good exponential decay
#epochs=50, totally my choice
#train_size = 6691 is how many training images were from the train_ds above

epochs = 50
train_size = 6691
batch_size = 128
initial_learning_rate = 0.1
final_learning_rate = 0.0001
learning_rate_decay_factor = (final_learning_rate / initial_learning_rate)**(1/epochs)
steps_per_epoch = int(train_size/batch_size)

In [ ]:
print(steps_per_epoch, learning_rate_decay_factor)

52 0.8709635899560806


In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=initial_learning_rate,
                decay_steps=steps_per_epoch,
                decay_rate=learning_rate_decay_factor,
                staircase=True)

#lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#    initial_learning_rate,
#    decay_steps=100000,
#    decay_rate=0.96,
#    staircase=True)

In [ ]:
#fairly confident on sparse categorical crossentropy, no idea on the other ones
model_reduced.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule),
              metrics=['accuracy'])

In [ ]:
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
history = model_reduced.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs, 
  callbacks=[model_checkpoint_callback, early_stopping_callback]
)

Epoch 1/50
210/210 [==============================] - 3230s 15s/step - loss: 77.0797 - accuracy: 0.0275 - val_loss: 150.2196 - val_accuracy: 0.0036
Epoch 2/50
210/210 [==============================] - 112s 533ms/step - loss: 40.8846 - accuracy: 0.1575 - val_loss: 97.8366 - val_accuracy: 0.0167
Epoch 3/50
210/210 [==============================] - 112s 532ms/step - loss: 22.3389 - accuracy: 0.2844 - val_loss: 34.4139 - val_accuracy: 0.1238
Epoch 4/50
196/210 [===========================>..] - ETA: 5s - loss: 15.9402 - accuracy: 0.3584

In [ ]:
https://keras.io/api/optimizers/learning_rate_schedules/exponential_decay/

#TODO - add in checkpoint so it saves the best one
https://keras.io/api/callbacks/model_checkpoint/

model.compile(loss=..., optimizer=...,
              metrics=['accuracy'])

EPOCHS = 10
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Model weights are saved at the end of every epoch, if it's the best seen
# so far.
model.fit(epochs=EPOCHS, callbacks=[model_checkpoint_callback])

# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)